# CUDA Codegen Testing Playground

A "playfield" for exploring whether the **experimental CUDA codegen** correctly identifies **constant (`const`) arguments** or whether its behavior is at least 
similar to the **legacy codegen**.

**Why does this matters?** Using `const` in CUDA can lead (and usually does lead) to **better performance** by enabling compiler optimizations.  

This notebook helps verify that constant arguments are being properly recognized.

In [1]:
# import all modules needed at once
import dace
import cupy as cp
import numpy as np
from IPython.display import Code
from typing import Optional

from dace import SDFG, properties
from dace.config import Config
from dace.transformation import pass_pipeline as ppl, transformation
from dace.sdfg import nodes
from dace import dtypes
from dace.transformation.passes.gpustream_scheduling import NaiveGPUStreamScheduler

Look which codegen we are currently using (legay or experimental):

In [2]:
usedCodegen = Config.get('compiler', 'cuda', 'implementation')
usedCodegen

'experimental'

In [3]:
N = dace.symbol('N')

@dace.program
def vector_copy_dyn_sizes(A: dace.float64[N] @ dace.dtypes.StorageType.GPU_Global, B: dace.float64[N] @ dace.dtypes.StorageType.GPU_Global):
    for i in dace.map[0:N] @ dace.dtypes.ScheduleType.GPU_Device:
            A[i] = B[i]

sdfg = vector_copy_dyn_sizes.to_sdfg()

In [4]:
Code(sdfg.generate_code()[0].clean_code, language='cpp')

/home/berkay/master-thesis/dace/dace/codegen/targets/experimental_cuda.py:1323: UserWarning: No `gpu_block_size` property specified on map "vector_copy_dyn_sizes_5". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


/* DaCe AUTO-GENERATED FILE. DO NOT MODIFY */
#include <dace/dace.h>
#include "../../include/hash.h"

struct vector_copy_dyn_sizes_state_t {
    dace::cuda::Context *gpu_context;
};

DACE_EXPORTED void __dace_runkernel_vector_copy_dyn_sizes_5_0_0_0(vector_copy_dyn_sizes_state_t *__state, double * __restrict__ A, const double * __restrict__ B, int N);
void __program_vector_copy_dyn_sizes_internal(vector_copy_dyn_sizes_state_t*__state, double * __restrict__ A, double * __restrict__ B, int N)
{

    {

        __dace_runkernel_vector_copy_dyn_sizes_5_0_0_0(__state, A, B, N);
        {

            ///////////////////
            DACE_GPU_CHECK(cudaStreamSynchronize(nullptr));
            ///////////////////

        }

    }
}

DACE_EXPORTED void __program_vector_copy_dyn_sizes(vector_copy_dyn_sizes_state_t *__state, double * __restrict__ A, double * __restrict__ B, int N)
{
    __program_vector_copy_dyn_sizes_internal(__state, A, B, N);
}
DACE_EXPORTED int __dace_init_experimental_cuda(vector_copy_dyn_sizes_state_t *__state, int N);
DACE_EXPORTED int __dace_exit_experimental_cuda(vector_copy_dyn_sizes_state_t *__state);

DACE_EXPORTED vector_copy_dyn_sizes_state_t *__dace_init_vector_copy_dyn_sizes(int N)
{
    int __result = 0;
    vector_copy_dyn_sizes_state_t *__state = new vector_copy_dyn_sizes_state_t;


    __result |= __dace_init_experimental_cuda(__state, N);

    if (__result) {
        delete __state;
        return nullptr;
    }
    return __state;
}

DACE_EXPORTED int __dace_exit_vector_copy_dyn_sizes(vector_copy_dyn_sizes_state_t *__state)
{
    int __err = 0;

    int __err_experimental_cuda = __dace_exit_experimental_cuda(__state);
    if (__err_experimental_cuda) {
        __err = __err_experimental_cuda;
    }
    delete __state;
    return __err;
}

In [5]:
Code(sdfg.generate_code()[1].clean_code, language='cpp')

#include <cuda_runtime.h>
#include <dace/dace.h>


struct vector_copy_dyn_sizes_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(vector_copy_dyn_sizes_state_t *__state, int N);
DACE_EXPORTED int __dace_exit_experimental_cuda(vector_copy_dyn_sizes_state_t *__state);



int __dace_init_experimental_cuda(vector_copy_dyn_sizes_state_t *__state, int N) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(0, 0);

    // Create cuda streams and events
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(vector_copy_dyn_sizes_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(vector_copy_dyn_sizes_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 0)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(vector_copy_dyn_sizes_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 0; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void __launch_bounds__(32) vector_copy_dyn_sizes_5_0_0_0(double * __restrict__ A, const double * __restrict__ B, int N)
{
    int i = (blockIdx.x * 32 + threadIdx.x);
    if (i < N) {
        {
            double __inp = B[i];
            double __out;

            ///////////////////
            // Tasklet code (assign_6_12)
            __out = __inp;
            ///////////////////

            A[i] = __out;
        }
    }
}


DACE_EXPORTED void __dace_runkernel_vector_copy_dyn_sizes_5_0_0_0(vector_copy_dyn_sizes_state_t *__state, double * __restrict__ A, const double * __restrict__ B, int N);
void __dace_runkernel_vector_copy_dyn_sizes_5_0_0_0(vector_copy_dyn_sizes_state_t *__state, double * __restrict__ A, const double * __restrict__ B, int N)
{


    if (((int_ceil(int_ceil(N, 1), 32)) <= 0)) {

        return;
    }

    void  *vector_copy_dyn_sizes_5_0_0_0_args[] = { (void *)&A, (void *)&B, (void *)&N };
    gpuError_t __err = cudaLaunchKernel( (void*)vector_copy_dyn_sizes_5_0_0_0, dim3(int_ceil(int_ceil(N, 1), 32), 1, 1), dim3(32, 1, 1), vector_copy_dyn_sizes_5_0_0_0_args, 0, nullptr
    );

    DACE_KERNEL_LAUNCH_CHECK(__err, "vector_copy_dyn_sizes_5_0_0_0", int_ceil(int_ceil(N, 1), 32), 1, 1, 32, 1, 1);
}

In [6]:
Code(sdfg.generate_code()[3].clean_code, language='cpp')

#include <cstdlib>
#include "../include/vector_copy_dyn_sizes.h"

int main(int argc, char **argv) {
    vector_copy_dyn_sizesHandle_t handle;
    int err;
    int N = 42;
    double * __restrict__ A = (double*) calloc(N, sizeof(double));
    double * __restrict__ B = (double*) calloc(N, sizeof(double));


    handle = __dace_init_vector_copy_dyn_sizes(N);
    __program_vector_copy_dyn_sizes(handle, A, B, N);
    err = __dace_exit_vector_copy_dyn_sizes(handle);

    free(A);
    free(B);


    return err;
}